In [1]:
# TODO: Fix use of arrays in the dictionaries to just values
import re
import ast
import sys
import time
import requests
import traceback
import pandas as pd
import numpy as np
import os.path
import urllib.request
import concurrent.futures
from os import listdir
from stem import Signal
from requests import get
from bs4 import BeautifulSoup
from datetime import datetime
from os.path import isfile, join
from stem.control import Controller

pickles_save_dir = "./pickles"

In [2]:
def get_tor_session():
    session = requests.session()
    # Tor uses the 9050 port as the default socks port
    session.proxies = {'http':  'socks5://127.0.0.1:9050',
                       'https': 'socks5://127.0.0.1:9050'}
    return session

def renew_connection():
    with Controller.from_port(port = 9051) as controller:
        controller.authenticate(password="password")
        controller.signal(Signal.NEWNYM)

In [59]:
if os.path.exists('./pickles/filtered_movies.pkl'):
    movies = pd.read_pickle("./pickles/filtered_movies.pkl")
else:
    title_basics = pd.read_csv("data/title.basics.tsv.gz", sep='\t')
    movies = title_basics[title_basics.titleType == 'movie']
    if not os.path.exists(pickles_save_dir):
        os.makedirs(pickles_save_dir)
    movies.to_pickle("{0}/filtered_movies.pkl".format(pickles_save_dir))
movies.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
14,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
238,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,\N,20,"Documentary,News,Sport"
552,tt0000335,movie,Soldiers of the Cross,Soldiers of the Cross,0,1900,\N,\N,"Biography,Drama"
825,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N
952,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Biography,Crime,Drama"


In [60]:
len(movies)

537487

In [ ]:
session = get_tor_session()
url = 'https://www.imdb.com/title/tt0120338/fullcredits'
data = get_data(session, url)
print(data)
print(len(data['cast']))

In [7]:
def get_data(session, url):
    info=[]
    
    r = session.get(url)
    r.raise_for_status()
    
    page_body = r.text
    soup = BeautifulSoup(page_body, 'html.parser')
    
    #tconst
    tconst=url.rsplit('/', 2)[-2]
    
    scraped_data = {
        "tconst": tconst,
        "cast": []
    }
    
    table = soup.find('table', class_='cast_list')
    if table is not None:
        atags = table.find_all('a')
        if atags is not None:
            for atag in atags:
                nconst_res = re.search('nm\d{7,8}', atag.get('href'))
                if nconst_res is not None:
                    scraped_data['cast'].append(nconst_res.group(0))
            
    # Drop duplicates
    scraped_data['cast'] = list(dict.fromkeys(scraped_data['cast']))
    scraped_data['count'] = len(scraped_data['cast'])
    return scraped_data

In [80]:
scraped_cast = pd.read_pickle("./pickles/castretries/complete_cast.pkl")

In [82]:
scraped_cast_nulls = scraped_cast[scraped_cast.astype(str)['cast'] == '[]']
movies = scraped_cast_nulls

In [38]:
# Set the start index and end index. Ip is renewed after each chunk of urls of 'step_size'. 
# After every chunk, result is written to a pickle. So if you need to stop the execution in the middle, note the
# index range of the last successfully written pickle file (from the printed logs) and use the remaining range
# for start and end index to resume from where you stopped.
postfix = 'tor'
s_idx = 0
e_idx = len(movies)
step_size = 4000

base_url = 'https://www.imdb.com/title/{0}/fullcredits'
urls=[]
for index, row in movies[s_idx:e_idx].iterrows():
    urls.append(base_url.format(row['tconst']))

used_ips = []
failed_urls = []

with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
    session = get_tor_session()
    chunk_indeces = np.arange(0, len(urls), step_size)
    
    for idx, start in enumerate(chunk_indeces, start=1):
        start_t = time.time()
        finals = []
        renew_connection()
        session = get_tor_session()
        new_ip = ast.literal_eval(session.get("http://httpbin.org/ip").text)["origin"].split(",")[0]
        while new_ip in used_ips:
            print("Renewed IP {0} already used. Waiting 5s to renew...".format(new_ip))
            time.sleep(5)
            renew_connection()
            session = get_tor_session()
            new_ip = ast.literal_eval(session.get("http://httpbin.org/ip").text)["origin"].split(",")[0]
        used_ips.append(new_ip)
        
        url_chunk = urls[start:start + step_size] + failed_urls
        failed_urls = []
        
        future_to_url = {executor.submit(get_data, session, url): url for url in url_chunk}
        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                data = future.result()
                finals.append(data)
            except Exception as exc:
                failed_urls.append(url.strip("'"))
                print('%s generated an exception: %s. Added to retry list.' % (url, exc))
                exc_info = sys.exc_info()
                traceback.print_exception(*exc_info)
        
        concurrent.futures.wait(
            list(future_to_url.keys()), 
            timeout=None, 
            return_when=concurrent.futures.ALL_COMPLETED)
        end_t = time.time()
        print("chunk {0} of {1} completed with IP {2}. Took {3} seconds."\
              .format(idx, len(chunk_indeces), new_ip, end_t - start_t))   
        cast_df = pd.DataFrame(data=finals)
        cast_pickles_savedir = "{0}/castretries/nullrecheck".format(pickles_save_dir)
        if not os.path.exists(cast_pickles_savedir):
            os.makedirs(cast_pickles_savedir)
        pickle_path = "{4}/{0}_{1}-{2}_{3}.pkl".format(idx, start, start + step_size, postfix, cast_pickles_savedir)
        cast_df.to_pickle(pickle_path)
        print("Wrote pickle {0} with {1} rows".format(pickle_path, len(cast_df)))
                
if len(failed_urls) > 0:
    print('\n\n\n\n!!!**********************************!!!')
    print('Failed to retrieve the following URLs.\n')
    print(failed_urls)
    print('\n!!!**********************************!!!')
    print('Note: Run the following cell to retry the failed urls')
else:
    print("\n\n\n\n!!!**********************************!!!\n")
    print("Scraping succeeded!")
    print('\n!!!**********************************!!!')
    print('''Note: Next, run the script that combine all pickles''')

https://www.imdb.com/title/tt10079846/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt10079846/fullcredits. Added to retry list.
https://www.imdb.com/title/tt0772169/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt0772169/fullcredits. Added to retry list.
https://www.imdb.com/title/tt0772169/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt0772169/fullcredits. Added to retry list.
https://www.imdb.com/title/tt10079846/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt10079846/fullcredits. Added to retry list.
https://www.imdb.com/title/tt10086905/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt10086905/fullcredits. Added to retry list.
https://www.imdb.com/title/tt10089485/fullcredits generated an exception: 404 Client Erro

https://www.imdb.com/title/tt4193114/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt4193114/fullcredits. Added to retry list.
https://www.imdb.com/title/tt4120944/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt4120944/fullcredits. Added to retry list.
https://www.imdb.com/title/tt4967582/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt4967582/fullcredits. Added to retry list.
https://www.imdb.com/title/tt4682042/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt4682042/fullcredits. Added to retry list.
https://www.imdb.com/title/tt4991770/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt4991770/fullcredits. Added to retry list.
https://www.imdb.com/title/tt5132990/fullcredits generated an exception: 503 Server Error: Serv

https://www.imdb.com/title/tt6052960/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt6052960/fullcredits. Added to retry list.
https://www.imdb.com/title/tt4905886/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt4905886/fullcredits. Added to retry list.
https://www.imdb.com/title/tt6251276/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt6251276/fullcredits. Added to retry list.
https://www.imdb.com/title/tt6449610/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt6449610/fullcredits. Added to retry list.
https://www.imdb.com/title/tt4651452/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt4651452/fullcredits. Added to retry list.
https://www.imdb.com/title/tt6361046/fullcredits generated an exc

https://www.imdb.com/title/tt6449598/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt6449598/fullcredits. Added to retry list.
https://www.imdb.com/title/tt6324788/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt6324788/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7168278/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt7168278/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7202342/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7202342/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7139086/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt7139086/fullcredits. Added to retry list.
https://www.imdb.com/title/tt6334244/fullcredits generated an exception: 40

https://www.imdb.com/title/tt7263172/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7263172/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7589524/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7589524/fullcredits. Added to retry list.
https://www.imdb.com/title/tt6835856/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt6835856/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7588822/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7588822/fullcredits. Added to retry list.
https://www.imdb.com/title/tt6568416/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt6568416/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7589532/fullcredits genera

...

https://www.imdb.com/title/tt7791242/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7791242/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7790888/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7790888/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7787004/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7787004/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7790844/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7790844/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7790310/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7790310/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7790888/fullcred

https://www.imdb.com/title/tt7792838/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7792838/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7792552/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7792552/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7792258/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7792258/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7787678/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7787678/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7792822/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7792822/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7789926/fullcred

https://www.imdb.com/title/tt7893578/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7893578/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7789328/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7789328/fullcredits. Added to retry list.
https://www.imdb.com/title/tt8708190/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt8708190/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7788554/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7788554/fullcredits. Added to retry list.
https://www.imdb.com/title/tt8059986/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt8059986/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7783816/fullcred

https://www.imdb.com/title/tt7790646/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7790646/fullcredits. Added to retry list.
https://www.imdb.com/title/tt8713190/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt8713190/fullcredits. Added to retry list.
https://www.imdb.com/title/tt8713568/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt8713568/fullcredits. Added to retry list.
https://www.imdb.com/title/tt8712914/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt8712914/fullcredits. Added to retry list.
https://www.imdb.com/title/tt8713000/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt8713000/fullcredits. Added to retry list.
https://www.imdb.com/title/tt8710418/fullcred

https://www.imdb.com/title/tt8174652/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt8174652/fullcredits. Added to retry list.
https://www.imdb.com/title/tt8708410/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt8708410/fullcredits. Added to retry list.
https://www.imdb.com/title/tt8129950/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt8129950/fullcredits. Added to retry list.
https://www.imdb.com/title/tt8111882/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt8111882/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7923364/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7923364/fullcredits. Added to retry list.
https://www.imdb.com/title/tt8174486/fullcred

In [39]:
len(failed_urls)

3175

In [40]:
'''
Run the following script only if there were any failed URLS at the end of the previous script execution.
Run this several times if you still see failures until there are none. But if some dont succeed even 
after multiple attempts, its probably a problem on imdb side and we can just ignore them.
'''
urls = failed_urls

if len(urls) > 0:
    retry_failed_urls = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=150) as executor:
        session = get_tor_session()
        chunk_indeces = np.arange(0, len(urls), step_size)

        for idx, start in enumerate(chunk_indeces, start=1):
            start_t = time.time()
            finals = []
            renew_connection()
            session = get_tor_session()
            new_ip = ast.literal_eval(session.get("http://httpbin.org/ip").text)["origin"].split(",")[0]
            while new_ip in used_ips:
                print("Renewed IP {0} already used. Waiting 5s to renew...".format(new_ip))
                time.sleep(5)
                renew_connection()
                session = get_tor_session()
                new_ip = ast.literal_eval(session.get("http://httpbin.org/ip").text)["origin"].split(",")[0]
            used_ips.append(new_ip)

            url_chunk = urls[start:start + step_size] + retry_failed_urls
            retry_failed_urls = []

            future_to_url = {executor.submit(get_data, session, url): url for url in url_chunk}
            for future in concurrent.futures.as_completed(future_to_url):
                url = future_to_url[future]
                try:
                    data = future.result()
                    finals.append(data)
                except Exception as exc:
                    retry_failed_urls.append(url.strip("'"))
                    print('%s generated an exception: %s. Added to retry list.' % (url, exc))

            concurrent.futures.wait(list(future_to_url.keys()), timeout=None, return_when=concurrent.futures.ALL_COMPLETED)
            end_t = time.time()
            print("chunk {0} of {1} completed with IP {2}. Took {3} seconds.".format(idx, len(chunk_indeces), new_ip, end_t - start_t))   
            cast_df = pd.DataFrame(data=finals)
            if len(cast_df) > 0:
                cast_pickles_savedir = "{0}/castretries/nullrecheck".format(pickles_save_dir)
                if not os.path.exists(cast_pickles_savedir):
                    os.makedirs(cast_pickles_savedir)
                pickle_path = "{4}/{0}_{1}-{2}_{3}_retry.pkl".format(idx, start, start + step_size, postfix, cast_pickles_savedir)
                cast_df.to_pickle(pickle_path)
                print("Wrote pickle {0} with {1} rows".format(pickle_path, len(cast_df)))

        if len(retry_failed_urls) > 0:
            print('\n\n\n\n!!!**********************************!!!')
            print('Retry failed with the following URLs.\n')
            print(retry_failed_urls)
            print('\n!!!**********************************!!!')
            print('Note: Run this script several times to see if all succeed.',
            'If some dont, it probably means they wont ever. so just give them up.')
        else:
            print("\n\n\n\n!!!**********************************!!!\n")
            print("Retry succeeded!")
            print('\n!!!**********************************!!!')
            print('''Note: Run the following script to combine all pickles''')

https://www.imdb.com/title/tt9335840/fullcredits generated an exception: SOCKSHTTPSConnectionPool(host='www.imdb.com', port=443): Max retries exceeded with url: /title/tt9335840/fullcredits (Caused by NewConnectionError('<urllib3.contrib.socks.SOCKSHTTPSConnection object at 0x1b7e37ef0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',)). Added to retry list.
https://www.imdb.com/title/tt9118844/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt9118844/fullcredits. Added to retry list.
https://www.imdb.com/title/tt8768374/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt8768374/fullcredits. Added to retry list.
https://www.imdb.com/title/tt8768374/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt8768374/fullcredits. Added to retry list.
https://www.imdb.com/title/tt9335840/fullcredits 

https://www.imdb.com/title/tt5835452/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt5835452/fullcredits. Added to retry list.
https://www.imdb.com/title/tt2592116/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt2592116/fullcredits. Added to retry list.
https://www.imdb.com/title/tt3213604/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt3213604/fullcredits. Added to retry list.
https://www.imdb.com/title/tt3277562/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt3277562/fullcredits. Added to retry list.
https://www.imdb.com/title/tt3951140/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt3951140/fullcredits. Added to retry list.
https://www.imdb.com/title/tt5134562/fullcredits generated an exception: 404 Client Error: Not 

https://www.imdb.com/title/tt2823068/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt2823068/fullcredits. Added to retry list.
https://www.imdb.com/title/tt2977174/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt2977174/fullcredits. Added to retry list.
https://www.imdb.com/title/tt6045460/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt6045460/fullcredits. Added to retry list.
https://www.imdb.com/title/tt5928906/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt5928906/fullcredits. Added to retry list.
https://www.imdb.com/title/tt3104664/fullcredits generated an exception: 404 Client Error: Not Found for url: https://www.imdb.com/title/tt3104664/fullcredits. Added to retry list.
https://www.imdb.com/title/tt4850734/fullcredits generated an exception: 404 Client Error: Not 

https://www.imdb.com/title/tt7683166/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7683166/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7673416/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7673416/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7684416/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7684416/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7678456/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7678456/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7683892/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7683892/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7673602/fullcred

https://www.imdb.com/title/tt7683350/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7683350/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7681562/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7681562/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7686152/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7686152/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7687372/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7687372/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7688836/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7688836/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7683696/fullcred

...

https://www.imdb.com/title/tt7791090/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7791090/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7787524/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7787524/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7787628/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7787628/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7790048/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7790048/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7784302/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7784302/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7788938/fullcred

https://www.imdb.com/title/tt7787878/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7787878/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7792722/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7792722/fullcredits. Added to retry list.
https://www.imdb.com/title/tt8710300/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt8710300/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7793096/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7793096/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7785020/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7785020/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7786852/fullcred

https://www.imdb.com/title/tt8710596/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt8710596/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7790918/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7790918/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7885688/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7885688/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7793124/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7793124/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7893578/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7893578/fullcredits. Added to retry list.
https://www.imdb.com/title/tt8710698/fullcred

https://www.imdb.com/title/tt8710598/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt8710598/fullcredits. Added to retry list.
https://www.imdb.com/title/tt8712834/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt8712834/fullcredits. Added to retry list.
https://www.imdb.com/title/tt8713190/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt8713190/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7792268/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt7792268/fullcredits. Added to retry list.
https://www.imdb.com/title/tt8713272/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt8713272/fullcredits. Added to retry list.
https://www.imdb.com/title/tt7793450/fullcred

https://www.imdb.com/title/tt8111882/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt8111882/fullcredits. Added to retry list.
https://www.imdb.com/title/tt8174486/fullcredits generated an exception: 503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt8174486/fullcredits. Added to retry list.
chunk 1 of 1 completed with IP 195.189.96.147. Took 96.44786787033081 seconds.
Wrote pickle ./pickles/castretries/1_0-4000_tor_retry.pkl with 1529 rows




!!!**********************************!!!
Retry failed with the following URLs.

['https://www.imdb.com/title/tt9335840/fullcredits', 'https://www.imdb.com/title/tt9118844/fullcredits', 'https://www.imdb.com/title/tt8768374/fullcredits', 'https://www.imdb.com/title/tt8768374/fullcredits', 'https://www.imdb.com/title/tt9335840/fullcredits', 'https://www.imdb.com/title/tt9412492/fullcredits', 'https://www.imdb.com/title/tt9174618/fullcredits', 'https://www.imdb

In [10]:
# Combine the created pickles into one Dataframe

dir_path = "{0}/casttor".format(pickles_save_dir)
# Ignore any sytem files starting with . or folders if any
pickles = [f for f in listdir(dir_path) if isfile(join(dir_path, f)) and not f.startswith('.')]
dfs = list(map(lambda x: pd.read_pickle("{0}/{1}".format(dir_path,x)), pickles))
combined_df = pd.concat(dfs)
combined_df.reset_index(drop=True, inplace=True)
combined_df.to_pickle("./pickles/complete_cast{0}.pkl".format(e_idx))
len(combined_df)
with pd.option_context('display.max_rows', 50000, 'display.max_columns', 20):
    display(combined_df)


,tconst,cast,count
0,tt0000502,"[nm0215752, nm0252720]",2
1,tt0000009,"[nm0063086, nm0183823, nm1309758]",3
2,tt0000009,"[nm0063086, nm0183823, nm1309758]",3
3,tt0000574,"[nm0846887, nm0846894, nm3002376, nm0170118, n...",15
4,tt0000574,"[nm0846887, nm0846894, nm3002376, nm0170118, n...",15
5,tt0000147,"[nm0179163, nm0280615, nm4082222, nm4081458, n...",5
6,tt0000502,"[nm0215752, nm0252720]",2
7,tt0000679,"[nm0000875, nm0122665, nm0933446, nm2924919, n...",34
8,tt0000675,[],0
9,tt0000679,"[nm0000875, nm0122665, nm0933446, nm2924919, n...",34


In [4]:
# dir_path = "./pickles/castretries/more"
# pickles = [f for f in listdir(dir_path) if isfile(join(dir_path, f)) and not f.startswith('.')]
# dfs = list(map(lambda x: pd.read_pickle("{0}/{1}".format(dir_path,x)), pickles))
# combined_df = pd.concat(dfs)
# combined_df.reset_index(drop=True, inplace=True)
# combined_df.to_pickle("./pickles/castretries/complete_cast.pkl")
# len(combined_df)

543452

In [18]:
scraped_cast = pd.read_pickle("./pickles/castretries/complete_cast.pkl")

In [19]:
len(scraped_cast)

539944

In [22]:
scraped_cast.astype(str).duplicated(subset='tconst').value_counts()

False    537487
True       2457
dtype: int64

In [ ]:
with pd.option_context('display.max_rows', 50000, 'display.max_columns', 20):
    display(scraped_cast[scraped_cast.astype(str).duplicated(subset='tconst', keep=False)].sort_values('tconst'))

In [ ]:
duplicates = scraped_cast[scraped_cast.astype(str).duplicated(subset='tconst', keep=False)].sort_values('tconst')
duplicates['cast'] = duplicates['cast'].apply(lambda x: ','.join(map(str, x)))
duplicates['idx'] = duplicates.index
with pd.option_context('display.max_rows', 50000, 'display.max_columns', 20):
    display(duplicates)

In [58]:
scraped_cast_uniq.to_pickle("./pickles/castretries/complete_cast.pkl")

In [61]:
scraped_cast = scraped_cast_uniq

In [63]:
scraped_tconst = set(scraped_cast.tconst)
movies_tconst = set(movies.tconst)

In [64]:
scraped_tconst.difference(movies_tconst)

set()

In [65]:
movies_tconst.difference(scraped_tconst)

set()

In [68]:
len(scraped_cast)

537487

In [69]:
len(movies)

537487

In [78]:
scraped_cast_nulls = scraped_cast[scraped_cast.astype(str)['cast'] == '[]']
with pd.option_context('display.max_rows', 50000, 'display.max_columns', 20):
    display(scraped_cast_nulls)

,tconst,cast,count
6,tt0790470,[],0
21,tt0788172,[],0
31,tt0788166,[],0
35,tt0790625,[],0
36,tt0790614,[],0
...,...,...,...
537466,tt6408846,[],0
537469,tt6409602,[],0
537470,tt6409158,[],0
537475,tt6409478,[],0
